# Code Summarization with Deep Reinforcement Learning
Paper[1] is linked: [Improving Code Summarization with Deep Reinforcement Learning](https://arxiv.org/abs/1811.07234)  
We're attempting to add concepts from Paper[2] [Unsupervised Translation of Programming Languages](https://arxiv.org/pdf/2006.03511)  
How? Not sure...  
I'm thinking of using their masking technique to finetune the product of Paper[1]. Essentially, with a given

# 1. Load Data

In [6]:
import pandas as pd
import os
cache_dir = 'E:\\.cache\\huggingface\\'
os.environ['HF_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir
import code_search_net
import datasets
from nltk.stem import PorterStemmer
import ast
import re

In [2]:
# You can use the below line instead of you don't want to load locally.
# BUt I havne't figured out to only choose certain datasets, I think its the uploader's fault
# Total is like 10+GB's
# d = datasets.load_dataset('code-search-net/code_search_net')

In [2]:
# What language you used for the data, otherwise default to all
dl_manager = datasets.DownloadManager('python')
csn = code_search_net.CodeSearchNet(config_name='python')
# You can remove the loc variable. My c:drive has no space so I did this instead.
generators = csn._split_generators(dl_manager, loc = 'E:\\.cache\\huggingface\\')

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
raw_data = []
for i, data in csn._generate_examples(generators[0].gen_kwargs['filepaths'][:1]):
    raw_data.append(data)

In [ ]:
# Of interest:
# func_code_tokens
# func_documentation_tokens
# possibly func_path_in_repository?
print(len(raw_data))
raw_data[0].keys()


30000


dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])

# 2. Create Inputs
"To convert code into sequential text, we tokenize the code by `{. , ” ’ : ; ) ( ! (space)}`, which  has been used in [8].  
We tokenize the comment by {(space)}" - Paper

In [7]:
stemmer = PorterStemmer()
def token_code(func_code_string):
    stripped = re.sub('"""[\s\S]*"""', '', func_code_string.lower()) # apaprently code has the entire docstring in there too
    no_comment = re.sub('#.*', '', stripped) # We should base it off just the comments
    tokens = re.split('[ \[\]{}\.,:;\)\(!\)=(\\n)(\\t)"\']', no_comment) # split appropaite, then next line remove ''
    return [stemmer.stem(x) for x in filter(None, tokens)], no_comment # https://stackoverflow.com/questions/30933216/split-by-regex-without-resulting-empty-strings-in-python
print(token_code(raw_data[0]['func_code_string']))
def token_comm(func_doc_string):
    stripped = re.sub('[\[\](\\n)|,:\.─├│└(\\t)"\']', ' ', func_doc_string.lower())
    tokens = re.split(" ", stripped)
    return [stemmer.stem(x) for x in filter(None, tokens)] # https://stackoverflow.com/questions/30933216/split-by-regex-without-resulting-empty-strings-in-python
print(token_comm(raw_data[0]['func_documentation_string']))

(['def', 'train', 'train_dir', 'model_save_path', 'none', 'n_neighbor', 'none', 'knn_algo', 'ball_tre', 'verbos', 'fals', 'x', 'y', 'for', 'class_dir', 'in', 'os', 'listdir', 'train_dir', 'if', 'not', 'os', 'path', 'isdir', 'os', 'path', 'join', 'train_dir', 'class_dir', 'continu', 'for', 'img_path', 'in', 'image_files_in_fold', 'os', 'path', 'join', 'train_dir', 'class_dir', 'imag', 'face_recognit', 'load_image_fil', 'img_path', 'face_bounding_box', 'face_recognit', 'face_loc', 'imag', 'if', 'len', 'face_bounding_box', '1', 'if', 'verbos', 'print', 'imag', 'not', 'suitabl', 'for', 'train', 'format', 'img_path', 'didn', 't', 'find', 'a', 'face', 'if', 'len', 'face_bounding_box', '<', '1', 'els', 'found', 'more', 'than', 'one', 'face', 'els', 'x', 'append', 'face_recognit', 'face_encod', 'imag', 'known_face_loc', 'face_bounding_box', '0', 'y', 'append', 'class_dir', 'if', 'n_neighbor', 'is', 'none', 'n_neighbor', 'int', 'round', 'math', 'sqrt', 'len', 'x', 'if', 'verbos', 'print', 'chos

## 2a. Vocab Creation
- Reparse func_code_string into tokens
- Might respase func_documaetnation_string
- lower case

In [8]:
vocab_code = {'<SOS>' : 0, '<EOS>': 1} # Is this necessary?
max_code = 0
i = 1
vocab_comm = {'<SOS>' : 0, '<EOS>': 1}
max_comm = 0
j = 1
for f in raw_data:
    # I think we will need to preprocess out own toeksn btu for right now prrof of concenpt:
    f['func_code_tokens'], f['func_code_string'] = token_code(f['func_code_string']) # replace with out tokens
    f['func_code_tokens'] = ['<SOS>'] + f['func_code_tokens'] + ['<EOS>']
    for code_token in f['func_code_tokens']:
        if code_token not in vocab_code.keys():
            vocab_code[code_token] = i
            i += 1
    max_code = max(max_code, len(f['func_code_tokens']))
    f['func_documentation_tokens'] =  ['<SOS>'] + token_comm(f['func_documentation_string']) + ['<EOS>']
    for comm_token in f['func_documentation_tokens']:
        if comm_token not in vocab_comm.keys():
            vocab_comm[comm_token] = j
            j += 1
    max_comm = max(max_comm, len(f['func_documentation_tokens']))
print('Vocab / Max Sentence Length')
print(len(vocab_code), max_code, ' | ', len(vocab_comm), max_comm)


Vocab / Max Sentence Length
124108 6184  |  58233 4959


In [9]:
# You can see some of these ar wrong which means we'll have to probably preprocess it outselves.
vocab_code

{'<SOS>': 0,
 '<EOS>': 1,
 'def': 1,
 'train': 2,
 'train_dir': 3,
 'model_save_path': 4,
 'none': 5,
 'n_neighbor': 6,
 'knn_algo': 7,
 'ball_tre': 8,
 'verbos': 9,
 'fals': 10,
 'x': 11,
 'y': 12,
 'for': 13,
 'class_dir': 14,
 'in': 15,
 'os': 16,
 'listdir': 17,
 'if': 18,
 'not': 19,
 'path': 20,
 'isdir': 21,
 'join': 22,
 'continu': 23,
 'img_path': 24,
 'image_files_in_fold': 25,
 'imag': 26,
 'face_recognit': 27,
 'load_image_fil': 28,
 'face_bounding_box': 29,
 'face_loc': 30,
 'len': 31,
 '1': 32,
 'print': 33,
 'suitabl': 34,
 'format': 35,
 'didn': 36,
 't': 37,
 'find': 38,
 'a': 39,
 'face': 40,
 '<': 41,
 'els': 42,
 'found': 43,
 'more': 44,
 'than': 45,
 'one': 46,
 'append': 47,
 'face_encod': 48,
 'known_face_loc': 49,
 '0': 50,
 'is': 51,
 'int': 52,
 'round': 53,
 'math': 54,
 'sqrt': 55,
 'chose': 56,
 'automat': 57,
 'knn_clf': 58,
 'neighbor': 59,
 'kneighborsclassifi': 60,
 'algorithm': 61,
 'weight': 62,
 'distanc': 63,
 'fit': 64,
 'with': 65,
 'open': 66,
 

In [10]:
vocab_comm

{'<SOS>': 0,
 '<EOS>': 1,
 'train': 1,
 'a': 2,
 'k-nearest': 3,
 'neighbor': 4,
 'classifi': 5,
 'for': 6,
 'face': 7,
 'recognit': 8,
 'param': 9,
 'train_dir': 10,
 'directori': 11,
 'that': 12,
 'contain': 13,
 'sub-directori': 14,
 'each': 15,
 'known': 16,
 'person': 17,
 'with': 18,
 'it': 19,
 'name': 20,
 'view': 21,
 'in': 22,
 'sourc': 23,
 'code': 24,
 'to': 25,
 'see': 26,
 'exampl': 27,
 'tree': 28,
 'structur': 29,
 '<train_dir>/': 30,
 '<person1>/': 31,
 '<somename1>': 32,
 'jpeg': 33,
 '<somename2>': 34,
 '<person2>/': 35,
 'model_save_path': 36,
 'option': 37,
 'path': 38,
 'save': 39,
 'model': 40,
 'on': 41,
 'disk': 42,
 'n_neighbor': 43,
 'number': 44,
 'of': 45,
 'weigh': 46,
 'classif': 47,
 'chosen': 48,
 'automat': 49,
 'if': 50,
 'not': 51,
 'specifi': 52,
 'knn_algo': 53,
 'underli': 54,
 'data': 55,
 'support': 56,
 'knn': 57,
 'default': 58,
 'is': 59,
 'ball_tre': 60,
 'verbos': 61,
 'return': 62,
 'wa': 63,
 'the': 64,
 'given': 65,
 'recogn': 66,
 'imag

## 2b. Create AST
ASTs to binary trees by the following two steps which have been adopted in [28]:  
- a) Split nodes with more than 2 children, generate a new right child together with the old left child as its children, and then put all children except the leftmost as the children of this new node.   
Repeat this operation in a top-down way until only nodes with 0, 1, 2 children left;
- b) Combine nodes with 1 child with its child.  

(from the paper)

In [14]:
def parse_tree(lines):
    #replace the DCNL & DCSP symbols
    #need to deal with different indentations& spacings. The authors used DCSP for both '\t' & whitespace

    lines=lines.replace(' DCNL ',' \n')
    lines=lines.replace(' DCSP ',' \t')
    lines=lines.replace('DCSP ',' \t')

    #create temporary python file for every line (function) in input file
    tree = ast.parse(lines)
    temp_ast=ast.dump(tree)
    
    return temp_ast
def clean_data(line):

    clean = re.sub(r"""
            [,.;@#?!&$()='\\_`:>"%/{}*]+  # Accept one or more copies of punctuation
            \ *           # plus zero or more copies of a space,
            """,
            " ",          # and replace it with a single space
            line, flags=re.VERBOSE)
    #Manually handle cases not accepted by sub
    clean = clean.replace("[", "")
    clean = clean.replace("]", "")
    clean = clean.replace("-", "")
    # tokenize on white space
    line = clean.split()
    # convert to lower case
    line = [word.lower() for word in line]
    # store as string
    return line
    # remove empty strings

errors = []
for i, f in enumerate(raw_data):
    try:
        f['func_ast_string'] = parse_tree(f['func_code_string'])
        f['func_ast_tokens'] = clean_data(f['func_ast_string'])
    except Exception as e:
        #print(e)
        errors.append(i)
len(errors)


789

## 2d. Pack them into a data structure
We should form Train/Test Splits, I think the `raw_data` variable already holds this. Should be something of:  
```
inputs[0] = {  
'seq' : list of sequential representation of code (converted to ID from sectiion 2a, lists from 2c)
'ast' : list of ast representation of code (converted to ID from section 2a, AST from 2b.)
}
make sure we padding them, and have <SOS> <EOS>
```

# 3. Model Definition
Probably hardest bit, I got as many variables from the paper as I could.  
i think a lot of it will be similar to what we did in class projects (Lab 5)

## 3a. Model params

In [ ]:
# Taken from paper
batch_size = 64
hidden_size = 512
embedding_size = 512 #LSTM, Layers
lr = 0.001
skip_num = 0
code_vocab_size = len(vocab_code)
#code_max_len = max_code
comment_vocab_size = len(vocab_comm)
#comment_max_len = max_comm # This and max_c

## 3b. Actor Network

## 3c. Critic Network

## 3d. Training Loop?

# 4. Modifications 

# Solver test